# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.9 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
import numpy as np
import math
import torch
import torch.nn as nn
import json
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple, deque


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def parse_params(params_dir):
    with open(params_dir) as fp:
        params = json.load(fp)
    return params


class A2C_model(nn.Module):
    def __init__(self, params_dir, input_dim, act_size):
        super().__init__()
        self.input_dim = input_dim
        self.params = parse_params(params_dir)
        self.base = self.create_base(self.input_dim)
        self.mu = nn.Sequential(nn.Linear(self.params['hidden_dim'],
                                          act_size), nn.Tanh())
        self.var = nn.Sequential(nn.Linear(self.params['hidden_dim'],
                                           act_size),
                                 nn.Softplus(threshold=5))
        self.val = nn.Sequential(nn.Linear(self.params['hidden_dim'], 1),
                                 nn.ReLU())
        self.std = nn.Parameter(torch.ones(1, act_size))

    def create_base(self, input_dim):
        module_list = nn.ModuleList()
        layer = nn.Sequential()
        fc = nn.Linear(input_dim, self.params['hidden_dim'])
        layer.add_module(f"fc_layer_1", fc)
        # layer.add_module(f"bn_layer_1",
                         # nn.BatchNorm1d(self.params['hidden_dim']))
        layer.add_module(f"RELU_layer_1", nn.LeakyReLU())
        module_list.append(layer)
        input_dim = self.params['hidden_dim']
        for i in range(2, self.params['num_hidden_layers']+2):
            fc = nn.Linear(input_dim, self.params['hidden_dim'])
            layer.add_module(f"fc_layer_{i}", fc)
            # layer.add_module(f"bn_layer_{i}",
                             # nn.BatchNorm1d(self.params['hidden_dim']))
            layer.add_module(f"RELU_layer_{i}", nn.LeakyReLU())
        # print(module_list)
        return module_list

    def forward(self, states):
        x_ = states
        for m in self.base:
            x_ = m(x_)
        mu = self.mu(x_)
        # var = self.var(x_)
        # etp = (-(torch.log(2*math.pi*var) + 1)/2).mean()
        # std = torch.sqrt(var)
        # print(f"var : {var}")
        # print(f"stand deviation : {std}")
        val = self.val(x_)
        # print(f"state value :{val}")
        return mu, self.std, val


class Agent_A2c():
    def __init__(self, model, num_agents):
        self.model = model
        self.device = device
        self.num_agents = num_agents
        self.params = self.model.params
        self.optimizer = optim.Adam(self.model.parameters(),
                                    lr=self.params['lr'])

    def __call__(self, states):
        # mu, std, val, etp = self.model(states)
        mu, std, val = self.model(states)
        dist = torch.distributions.Normal(mu, std)
        # actions = torch.clamp(dist.sample(), -1, 1)
        actions = dist.sample()
        # print(f"actions  {actions}")
        
        log_prob = dist.log_prob(actions)
        # print(f"log_prob after dist:  {log_prob}")
        return actions, log_prob, val

    def step(self, experiences, env):
        '''
            experiences:
                list with n_steps_taken * [actions, rewards, log_probs,
                                           not_dones, state_values]:
                    actions (tensor: num agents * num actions)
                    rewards (list: size = num agents)
                    log_probs (tensor: num agents * num actions)
                    not_dones (np array: size = num agents)
                    state_values (list: size = num agents)
        '''
        
        # ENTROPY_BETA = 0.001
        actions, rewards, log_probs, not_dones, state_values = experiences
        rewards = torch.FloatTensor(rewards).transpose(0, 1).contiguous()
        state_values_last = state_values[-1].detach()
        # state_values_last = state_values[-1]
                                             
        processed_experience = [None]* len(experiences[0])
        for i in reversed(range(len(experiences[0]))):
            power = (len(experiences[0])-i)
            not_done_ = torch.FloatTensor(not_dones[i]).to(device).unsqueeze(1)
            discount_ = np.array([((self.params['gamma'] ** p )) for p in range(power)])
            discount_ = torch.FloatTensor(discount_).to(device).unsqueeze(1)
            # print(f"discount_ : {discount_}")
            # print(f"return before discount : {rewards[:, i:]}")
            # print(f"Discounted return : {torch.mm(rewards[:, i:].to(device), discount_) }")
            # print(f"Before discounted last return: {state_values_last}")
            # print(f"discounted last return: {self.params['gamma'] ** power * not_done_ * state_values_last}")
            # print(f" gamma : {self.params['gamma']}  power: {power}, mm: {torch.mm(rewards[:, i:].to(device), discount_)}")
            rewards_ = torch.mm(rewards[:, i:].to(device), discount_)
            advantage_v_ =  rewards_+ self.params['gamma'] ** power * not_done_ * state_values_last - - state_values[i]
            # print(f"returns_ : {returns_}")
            advantage_v_ = returns_ - state_values[i]
            # print(f"advantage : {advantage_v_}")
            # advantage_p_ = advantage_v_.detach()
            advantage_p_ = returns_ - state_values[i].detach()
            # advantage_p_ = returns_ - state_values[i]
            processed_experience[i] = [log_probs[i], advantage_v_, advantage_p_]
        log_prob, advantage_v, advantage_p = map(lambda x: torch.cat(x, dim=0), zip(*processed_experience))
        # print(f"log_prob : {log_prob}")
        policy_loss = -log_prob * advantage_p.mean()
        value_loss = 0.5 * (advantage_v).pow(2).mean()
        # entropy_loss = ENTROPY_BETA * torch.FloatTensor(etps).to(device)
        # print(f"policy_loss : {policy_loss.mean()}")
        # print(f"value_loss : {value_loss.mean()}")
        loss_v = policy_loss  +  0.5*value_loss
        # print(f"loss : {loss_v.mean()}")
        self.optimizer.zero_grad()
        if torch.isnan(loss_v).any():
            print('very bad')
            pass
        loss_v.mean().backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 5) # clip gradient
        self.optimizer.step()

    
    
class Experience():
    def __init__(self):
        self.actions = []
        self.rewards = []
        self.log_probs = []
        self.not_dones = []
        self.state_values = []
        # self.etp = []

    def add(self, actions, rewards, log_probs, not_dones, state_values):
        self.actions.append(actions)
        self.rewards.append(rewards)
        self.log_probs.append(log_probs)
        self.not_dones.append(not_dones)
        self.state_values.append(state_values)
        # self.etp.append(etp)

    def spit(self):
        return (self.actions, self.rewards, self.log_probs, self.not_dones,
                self.state_values)

In [5]:
for i in range(5-2,-1,-1):
    print(i)

3
2
1
0


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


In [6]:
params_dir = f"./params.txt"
torch.manual_seed(420)
model = A2C_model(params_dir, state_size, action_size).to(device)
agent = Agent_A2c(model, num_agents)

In [67]:
print(1)

1


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
actions = np.random.randn(num_agents, action_size)
actions = np.clip(actions, -1, 1)  
env_info = env.step(actions)[brain_name]           # send all actions to tne environment
next_states = env_info.vector_observations   

In [7]:
env_info = env.step(actions)[brain_name]  

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [19]:
actions_, log_prob_, state_values_, etp_ = agent(
                torch.FloatTensor(states).to(device))

In [17]:
a = [12]
a.append(13)
print(a)

[12, 13]


In [7]:
scores_window = deque(maxlen=100)
for i in range(200):
    env_info = env.reset(train_mode=True)[brain_name]
    states_ = env_info.vector_observations
    scores = np.zeros(num_agents)
    memories = Experience()
    done = [False] * num_agents
    steps = 0
    # while not np.any(done):
    while True:
        actions_, log_prob_, state_values_ = agent(
                torch.FloatTensor(states).to(device))
        env_info = env.step(actions_.detach().cpu().numpy())[brain_name]
        next_states_ = env_info.vector_observations
        rewards_ = env_info.rewards
        done = env_info.local_done
        if np.any(done):
        # if steps > 11:
            break
        not_done_ = (1 - np.array(done))
        memories.add(actions_, rewards_, log_prob_, not_done_, state_values_)
        steps += 1
        if steps % 5 == 0:
            experiences = memories.spit()
            agent.step(experiences, env)
            memories = Experience()
        states = next_states_
        scores += rewards_
        # print(scores)
    print(np.mean(scores))
    scores_window.append(np.mean(scores))
    if len(scores_window) > 100 and ((sum(scores_window) / len(scores_window)) > 30):
        print("solved!")
        break

0.614999986254
0.623499986064
0.657999985293
0.876999980398
0.423499990534
0.494999988936
0.79499998223
0.982999978028
0.864499980677
0.956999978609
0.993999977782
0.25099999439
0.0299999993294
0.150499996636
0.0374999991618
0.0134999996983
0.0239999994636
0.0179999995977
0.0179999995977
0.0319999992847
0.0
0.0274999993853
0.00499999988824
0.0109999997541
0.112999997474
0.0299999993294
0.0484999989159
0.0754999983124
0.0134999996983
0.0439999990165
0.02549999943
0.0319999992847
0.0259999994189
0.0449999989942
0.0504999988712
0.00749999983236
0.0619999986142
0.0119999997318
0.0449999989942
0.0449999989942
0.0854999980889
0.202999995463
0.0194999995641
0.02549999943
0.0299999993294
0.0874999980442
0.0149999996647
0.0544999987818
0.550999987684
0.628499985952
0.682499984745
0.839499981236
0.745499983337
0.818499981705
0.763499982934
0.707499984186
0.736999983527
0.695999984443
0.725999983773
0.797999982163
0.881499980297
0.890999980085
0.764999982901
0.765999982879
0.633999985829
0.704999

In [193]:
actions, rewards, log_probs, not_dones, state_values, etp_ = experiences

In [24]:
import math
ENTROPY_BETA = 0.001
actions, rewards, log_probs, not_dones, state_values, etps = experiences
rewards = torch.FloatTensor(rewards).transpose(0, 1).contiguous()
state_values_last = state_values[-1].detach()
processed_experience = [None]* len(experiences[0])
for i in reversed(range(len(experiences[0]))):
    power = (len(experiences[0])-i)
    not_done_ = torch.FloatTensor(not_dones[i]).to(device).unsqueeze(1)
    discount_ = np.array([((self.params['gamma'] ** p )) for p in range(power)])
    discount_ = torch.FloatTensor(discount_).to(device).unsqueeze(1)
    returns_ = torch.mm(rewards[:, i:].to(device), discount_) + self.params['gamma'] ** power * not_done_ * state_values_last
    advantage_v_ = returns_ - state_values[i]
    # advantage_p_ = advantage_v_.detach()
    advantage_p_ = advantage_v_.copy()
    processed_experience[i] = [log_probs[i], advantage_v_, advantage_p_]
log_prob, advantage_v, advantage_p = map(lambda x: torch.cat(x, dim=0), zip(*processed_experience))
policy_loss = -log_prob * advantage_p.mean()
value_loss = 0.5 * (advantage_v).pow(2).mean()
entropy_loss = ENTROPY_BETA * torch.FloatTensor(etps).to(device)

loss_v = policy_loss  + value_loss
loss_v.mean().backward()
optimizer.step()
print(advantage_v_)
print(advantage_p_)

        

NameError: name 'optimizer' is not defined

In [76]:
loss_v.mean()

tensor(1.00000e-03 *
       -1.8518)

In [60]:
rewards[:, i:].shape

torch.Size([20, 1])

In [26]:
torch.mm(discount_, rewards[:, i:])

RuntimeError: matrices expected, got 1D, 2D tensors at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/TH/generic/THTensorMath.c:2028

In [254]:
power = (len(experiences)-i)
self.params['gamma']**power

0.99

In [22]:
class dummy():
    def __init__(self):
        self.params = {'gamma': 0.99}
self = dummy()

In [209]:
rewards[:,:i+1].shape

torch.Size([20, 5])

In [206]:
a = torch.ones(20, 4)
b = torch.rand(4, 1)
print(b)
print(torch.mm(a, b))

tensor([[ 0.8306],
        [ 0.3578],
        [ 0.6825],
        [ 0.7754]])
tensor([[ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462],
        [ 2.6462]])


In [205]:
torch.mm(rewards[:,:i+1], discount)

tensor([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]])

In [170]:
discount = torch.FloatTensor(np.array([((self.params['gamma'] ** p ))for p in range(len(experiences)-i)])).to(device) 
print(discount)

tensor([ 1.0000,  0.9900,  0.9801,  0.9703,  0.9606])


In [159]:
torch.zeros(5) * 2.2

tensor([ 0.,  0.,  0.,  0.,  0.])

In [132]:
print(torch.FloatTensor(not_dones[i]).to(device).unsqueeze(1).shape)

torch.Size([20, 1])


In [106]:
class dummy():
    def __init__(self):
        self.params = {'gamma':0.99}
self = dummy()    

In [108]:
print(self.params['gamma'])

0.99


In [133]:
print(i)

4


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 